In [5]:
import mysql.connector
from tabulate import tabulate


def mysql_request(query, params=None):
    """Выполняет SQL-запрос и возвращает результат."""
    connection = None
    try:
        connection = mysql.connector.connect(
            host='localhost',
            port=3306,
            user='root',
            password='123',
            db='sakila',
        )
        with connection.cursor(dictionary=True) as cursor:
            cursor.execute(query, params)
            result = cursor.fetchall()
            return result

    except mysql.connector.Error as ex:
        print(f"Database error [{ex.errno}]: {ex.msg}")
        return None

    finally:
        if connection and connection.is_connected():
            connection.close()


def get_categories():
    """Получает список категорий."""
    query = "SELECT category_id, name FROM category;"
    return mysql_request(query)


def print_categories():
    """Выводит список категорий."""
    categories = get_categories()
    if categories:
        print("\nСписок категорий:")
        print(tabulate(categories, headers="keys", tablefmt="fancy_grid"))
    else:
        print("Ошибка при получении категорий.")


def get_films_of_category(category_id, operator=None, value=None):
    """Получает фильмы в категории, применяя фильтр по году выпуска (если указан)."""
    filter_clause = ""
    params = [category_id]

    if operator and value:
        filter_clause = f"AND f.release_year {operator} %s"
        params.append(value)

    query = f"""
        SELECT f.title, f.release_year, f.description,
               GROUP_CONCAT(CONCAT(a.first_name, ' ', a.last_name) SEPARATOR ', ') AS actors
        FROM category c
        JOIN film_category fc ON c.category_id = fc.category_id
        JOIN film f ON fc.film_id = f.film_id
        LEFT JOIN film_actor fa ON f.film_id = fa.film_id
        LEFT JOIN actor a ON fa.actor_id = a.actor_id
        WHERE c.category_id = %s {filter_clause}
        GROUP BY f.film_id
        LIMIT 10;
    """
    
    return mysql_request(query, params)


def format_actors_for_new_lines(actors):
    """Функция для добавления каждого актёра в новую строку."""
    if actors:
        actor_list = actors.split(', ')  
        formatted_actors = "\n".join(actor_list)  
        return formatted_actors
    return actors

def print_films(category_id, operator=None, value=None):
    res_film = get_films_of_category(category_id, operator, value)
    if res_film:
        print("\n Фильмы в выбранной категории:")
        for film in res_film:
            film['actors'] = format_actors_for_new_lines(film['actors'])
        print(tabulate(res_film, headers="keys", tablefmt="fancy_grid", 
                       maxcolwidths=[None, None, 40, None, 40]))  
    else:
        print("No data or error in your request")


def get_table_fields():
    """Получает список полей таблицы."""
    table_name = input('Введите имя таблицы: ').strip()
    query = f"DESCRIBE {table_name};"
    return mysql_request(query)


def print_table_fields():
    res_fields = get_table_fields()
    if res_fields:
        print("\n List of fields:")
        print(tabulate(res_fields, headers="keys", tablefmt="fancy_grid"))
    else:
        print("Error at getting fields")


def print_filter(category_id):
    """Запрашивает у пользователя фильтр по году выпуска и выводит фильмы."""
    valid_operators = ['=', '>', '<', '>=', '<=', '!=']
    use_filter = input("Хотите добавить фильтр по году выпуска? (y/n): ").strip().lower() == 'y'

    operator, value = None, None
    if use_filter:
        while True:
            operator = input(f"Выберите оператор ({', '.join(valid_operators)}): ").strip()
            if operator in valid_operators:
                break
            print("Ошибка: Введите корректный оператор!")

        while True:
            value = input("Введите значение года выпуска: ").strip()
            if value.isdigit():
                value = int(value)
                break
            print("Ошибка: Введите числовое значение!")

    print_films(category_id, operator, value)


def main():
    print("Добро пожаловать в интерфейс работы с базой данных!")
    print_categories()

    while True:
        category_id = input("\nВведите номер категории (или 'exit' для выхода): ").strip()
        if category_id.lower() == 'exit':
            break
        if category_id.isdigit():
            print_filter(int(category_id))
        else:
            print("Ошибка: Введите корректный номер категории!")


if __name__ == "__main__":
    main()

Добро пожаловать в интерфейс работы с базой данных!

Список категорий:
╒═══════════════╤═════════════╕
│   category_id │ name        │
╞═══════════════╪═════════════╡
│             1 │ Action      │
├───────────────┼─────────────┤
│             2 │ Animation   │
├───────────────┼─────────────┤
│             3 │ Children    │
├───────────────┼─────────────┤
│             4 │ Classics    │
├───────────────┼─────────────┤
│             5 │ Comedy      │
├───────────────┼─────────────┤
│             6 │ Documentary │
├───────────────┼─────────────┤
│             7 │ Drama       │
├───────────────┼─────────────┤
│             8 │ Family      │
├───────────────┼─────────────┤
│             9 │ Foreign     │
├───────────────┼─────────────┤
│            10 │ Games       │
├───────────────┼─────────────┤
│            11 │ Horror      │
├───────────────┼─────────────┤
│            12 │ Music       │
├───────────────┼─────────────┤
│            13 │ New         │
├───────────────┼─────────────┤
│


Введите номер категории (или 'exit' для выхода):  1
Хотите добавить фильтр по году выпуска? (y/n):  y
Выберите оператор (=, >, <, >=, <=, !=):  =
Введите значение года выпуска:  2006



 Фильмы в выбранной категории:
╒═════════════════════╤════════════════╤══════════════════════════════════════════╤═════════════════════╕
│ title               │   release_year │ description                              │ actors              │
╞═════════════════════╪════════════════╪══════════════════════════════════════════╪═════════════════════╡
│ AMADEUS HOLY        │           2006 │ A Emotional Display of a Pioneer And a   │ JOHNNY LOLLOBRIGIDA │
│                     │                │ Technical Writer who must Battle a Man   │ JULIA MCQUEEN       │
│                     │                │ in A Baloon                              │ VAL BOLGER          │
│                     │                │                                          │ KIRK JOVOVICH       │
│                     │                │                                          │ JAMES PITT          │
│                     │                │                                          │ PENELOPE CRONYN     │
├─────────────


Введите номер категории (или 'exit' для выхода):  exit
